# The 9-Qubit Code

In [ ]:
import sys
sys.path.append('..')   # the `general_qec` package sits above us

import numpy as np

# Importing required libraries
from general_qec.qec_helpers import *
from general_qec.gates import *
from general_qec.errors import *
from circuit_specific.nine_qubit_helpers import *

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit import user_config
from qiskit.quantum_info import partial_trace
from qiskit.extensions import UnitaryGate

# Visualization of circuits: Initializing backend simulators
qasm_sim = Aer.get_backend('qasm_simulator')
sv_sim = Aer.get_backend('statevector_simulator')

# Setting mpl as default drawer
%env QISKIT_SETTINGS {}
user_config.set_config('circuit_drawer', 'mpl')

It is important to note that when we measure the logical states of our qubit systems we will only actually "see" one of the states that make up our superposition terms of our qubit system state. This will happen in many notebooks but it is important to keep in mind that it is only for example's sake.

For example, if our state is $\vert\psi\rangle = \alpha\vert000\rangle + \beta\vert111\rangle$ we will either measure $\vert000\rangle$ or $\vert111\rangle$, never both! 

Another useful example that well come across is when we use ancilla qubits and measure them, collapsing our state. Lets say our state is $\vert\psi\rangle = \alpha_1\vert00001\rangle + \alpha_1\vert00010\rangle + \beta_2\vert11101\rangle + \beta_2\vert11110\rangle$. And lets say we measure our two ancilla qubits at the end (right side of our bit representation). The the states we would get are $\vert\psi\rangle = \gamma_1\vert00001\rangle + \lambda_1\vert11101\rangle$ or $\vert\psi\rangle = \gamma_2\vert00010\rangle + \lambda_2\vert11110\rangle$, but again never both!

**In our notebooks, we just display this information to show the current quantum state, although physically this would never be possible.**

## Contents
1. [Introduction](#introduction)
2. [Initializing the 9-qubit code](#initialization)
3. [Phase Error Detection and Correction](#phase)
4. [Bit Flip Error Detection and Correction](#bit)
5. [The Full 9-qubit code sequence](#full)

## 1. Introduction <a id='introduction'></a>

The 9-qubit code is very similar to the 3 qubit code since it is also a repitition code and it even uses 3 sets of the 3-qubit code in its construction. However, with the use of 9 qubits (11 total), the strength of the code increases. The 9-qubit code is able to correct for up to 3 bit flip errors and 1 phase error. Due to the high similarities, only the differences will be discussed in this section. For more information visit 02. 3 qubit Code Tutorial.

The key difference is the use of 9 qubits rather than just 3. This allows for the code to detect phase errors with the addition of Hadamard and CNOT gates applied to the 11 qubit system (which includes the ancilla). With this, there are an increased number of gates applied, and as seen when realistic error models are applied, this can lead to faster code failure in terms of iterations. 

In this tutorial, only the "ideal" number of errors are applied. Due to high computational times, the realistic error model is only implemented in a python file rather than a full tutorial. If you would like to see how the realistic error model is implemented on the 3 qubit code and the Steane code, please see 02c. 3 qubit logical T1 calculation and 03b. Steane code logical T1 calculation. The idea is the same for the 9 qubit code.

## 2. Initializing the 9-qubit code <a id='initialization'></a>

The 2 logical states used for this code are labeled $\vert0\rangle _L$ and $\vert1\rangle _L$ which can be defined as
$$ \vert0\rangle _L = \frac{1}{\sqrt{8}}(\vert000\rangle + \vert111\rangle)(\vert000\rangle + \vert111\rangle)(\vert000\rangle + \vert111\rangle) $$
$$ \vert1\rangle _L = \frac{1}{\sqrt{8}}(\vert000\rangle - \vert111\rangle)(\vert000\rangle - \vert111\rangle)(\vert000\rangle - \vert111\rangle) $$

The initialization process for the logical state of the 9 qubit code is the following.

In [ ]:
psi = QuantumRegister(1, '|ψ⟩')
ancilla = QuantumRegister(8, '|0⟩')

qc = QuantumCircuit(psi, ancilla)
qc.cx(0, 3)
qc.cx(0, 6)
qc.h(0)
qc.h(3)
qc.h(6)
qc.cx(0, 1)
qc.cx(3, 4)
qc.cx(6, 7)
qc.cx(0, 2)
qc.cx(3, 5)
qc.cx(6, 8)
qc.barrier()

qc.draw()

Example usage

In [ ]:
initial_psi = zero # |0>
# initial_psi = one # |1>
state = nine_qubit_initialize_logical_state(initial_psi)

print('Initialized 9 qubit logical state:')
print_state_info(state, 11)

### 3. Phase Error Detection and Correction <a id='phase'></a>

In order to detect phase errors in this circuit each block of 3 qubits is compared to the others. Since the logical states of the qubit (which are in the codespace) are in a certain configuration, this allows a comparison of phase such that if there is a single qubit in one of the blocks with an incorrect phase, the ancilla qubits will be triggered. This is done by putting the qubits in the $\vert+\rangle$ and $\vert-\rangle$ basis using Hadamard gates, such that when a CNOT gate is applied, the phase of the qubits will be "compared" using the ancillas. The circuit used to do this can be seen below, with the code implementation following.

In [ ]:
psi = QuantumRegister(1, '|ψ⟩')
ancilla = QuantumRegister(8, '|0⟩')
additional_ancilla = QuantumRegister(10, '|0⟩')
qc = QuantumCircuit(11)

# Hadamard gates on the left side (vertical arrangement)
for i in range(9):
    qc.h(i)

# Define the Z error correction circuit for Shor's code
qc.cx(0, 9)
qc.cx(1, 9)
qc.cx(2, 9)
qc.cx(3, 9)
qc.cx(4, 9)
qc.cx(5, 9)
qc.barrier()
qc.cx(3, 10)
qc.cx(4, 10)
qc.cx(5, 10)
qc.cx(6, 10)
qc.cx(7, 10)
qc.cx(8, 10)

# Add barrier gate after the error correction circuit
qc.barrier()

# Hadamard gates on the right side (vertical arrangement)
for i in range(9):
    qc.h(i)
qc.draw()

Example usage

In [ ]:
final_state = nine_qubit_phase_correction(state)

print('\n 9 qubit state with no errors:')
print_state_info(final_state, 11)

Applying a Z error to a random qubit in the system

In [ ]:
errored_state = random_qubit_z_error(state, qubit_range = [0,8])[0]

print('Errored 9 qubit state:')
print_state_info(errored_state, 11)

Correcting the Z error

In [ ]:
phase_corrected_state = nine_qubit_phase_correction(errored_state)

print('\nCorrected 9 qubit state:')
print_state_info(phase_corrected_state, 11)

###  4. Bit Flip Error Detection and Correction <a id='bit'></a>

As stated above, the bit flip error correction works exactly the same as the 3-qubit code. In this case each block is corrected individually using the 2 ancilla qubits, and after each block's correction the ancilla qubits are reset to ensure they are prepared for the next detection and correction. The circuit for this is shown below with the implementation after.

In [ ]:
# Define the X error correction circuit for Shor's code

psi = QuantumRegister(1, '|ψ⟩')
ancilla = QuantumRegister(8, '|0⟩')
additional_ancilla = QuantumRegister(10, '|0⟩')
qc = QuantumCircuit(11)


# The first block of three qubits 
qc.cx(0, 9)
qc.cx(1, 9)
qc.cx(0, 10)
qc.cx(2, 10)
qc.barrier()

# The second block of three qubits
qc.cx(3, 9)
qc.cx(4, 9)
qc.cx(3, 10)
qc.cx(5, 10)
qc.barrier()

# The third block of three qubits

qc.cx(6, 9)
qc.cx(7, 9)
qc.cx(6, 10)
qc.cx(8, 10)
qc.barrier()


qc.draw()

Implementing one random bit flip error on each block

In [ ]:
print_state_info(phase_corrected_state, 11)

print('\nThis cell is specific to block 1 ')
errored_state = random_qubit_x_error(errored_state, [0,2])[0]   # The [0,2] show qubits 1,2, and 3
print_state_info(errored_state, 11)

print('\nThis cell is specific to block 2 ')
errored_state = random_qubit_x_error(errored_state, [3,5])[0]   # The [3,5] show qubits 4,5, and 6
print_state_info(errored_state, 11)
print('\nThis cell is specific to block 3 ')

errored_state = random_qubit_x_error(errored_state, [6,8])[0]   # The [6,8] show qubits 7,8, and 9
print_state_info(errored_state, 11)

Example Usage of detecting the errors for each block

In [ ]:
# This cell is specific to block 1 
corrected_first_block_state = first_block(errored_state)
print_state_info(corrected_first_block_state, 11)
print('\n')
# This cell is specific to block 2
corrected_second_block_state = second_block(errored_state)
print_state_info(corrected_second_block_state, 11)
print('\n')
# This cell is specific to block 3
corrected_third_block_state = third_block(errored_state)
print_state_info(corrected_third_block_state, 11)

### 5. The Full 9-qubit code sequence <a id='full'></a>

The last step is to combine the initialization and detection (phase and bit) of the nine qubit code and implement them sequencially. This is done below:

In [ ]:
initial_state = nine_qubit_initialize_logical_state(np.array([1, 0]))
print('\ninitial_state: ')
print_state_info(initial_state, 11)

errored_state = random_qubit_z_error(initial_state, [0, 8])[0]
errored_state = random_qubit_x_error(errored_state, [0, 8])[0]
print('\nerrored_state: ')
print_state_info(errored_state, 11)

print('\ndetecting errors...')
corrected_state = full_nine_qubit_code(errored_state)

print('\ncorrected_state:')
print_state_info(corrected_state, 11)